In [1]:
# Dependencies
import sys
import time
import json
import requests
import importlib
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from config import gkey, zkey, zws_id, akey

In [2]:
# get list of zipcodes
%run ./write_zips.ipynb
import zipcodes
importlib.reload(zipcodes)
zip_codes = zipcodes.ziplist

Target city: atlanta
Search radius: 1


In [3]:
# get http request headers
def getHeaders():
    headers = {
        "apikey":akey,
        "accept":"application/json"
    }
    return headers


# GET /property/address via zip code
def fetch(url, params):
    headers = getHeaders()
    res = requests.get(url, headers=headers, params=params).json()
    return res


# show load bar and wait for next request
def wait(t):    
    sys.stdout.write('.')
    sys.stdout.flush()
    time.sleep(t)

In [ ]:
# get properties for each zip code (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?"
property_results = []
prop_types = ['RESIDENTIAL (NEC)', 'CONDOMINIUM', 
              'APARTMENT', 'CABIN', 'DUPLEX', 
              'MANUFACTURED HOME', 'MOBILE HOME',
              'TRIPLEX', 'TOWNHOUSE/ROWHOUSE']

sys.stdout.write('working..')

for z in zip_codes:
    for pt in prop_types:
        params = {
            "postalcode": z,
            "propertytype": pt,
            "pagesize": "100"
        }

        res = fetch(url, params)

        property_results.append(res)
        wait(10)

working................

In [ ]:
# get attomIds from property results
properties = []
attomIds = []

for p in property_results:
    for q in p['property']:
        properties.append(q)

props_df = pd.DataFrame(properties)
attomId_series = props_df['identifier']

for a in attomId_series:
    attomIds.append(a['attomId'])

In [ ]:
# get home values for each property (10 queries/min)
url = "https://search.onboard-apis.com/propertyapi/v1.0.0/property/basicprofile?"
hv_results = []

sys.stdout.write("working..")

for a in attomIds:
    params = {
        "attomId": a,
        "page":"1",
        "pagesize":"100"
    }

    idres = fetch(url, params)
    hv_results.append(idres)    
    wait(10)

In [ ]:
prop_res = pd.DataFrame(hv_results)
props = prop_res['property']
zcmkv = []

for i in range(len(props)):
    zcmkv.append({"zipcode":props[i][0]['address']['postal1'], "avg_mkt_val":int(props[i][0]['assessment']['market']['mktTtlValue'])})

zips_mktvals_df = pd.DataFrame(zcmkv).set_index('zipcode')
mkt_avg = zips_mktvals_df.groupby('zipcode').sum()/len(props)

mkt_avg_df = pd.DataFrame(mkt_avg)
mkt_avg_df = mkt_avg_df.reset_index()
mkt_avg_df.style.format({'avg_mkt_val':'${:,.2f}'})